example: http://www.silota.com/docs/recipes/sql-denormalized-dimension-table-modeling-materialized-views.html

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pj2-project").getOrCreate()

# https://sparkbyexamples.com/apache-hive/pyspark-save-dataframe-to-hive-table/
SparkSession.builder.enableHiveSupport()

23/05/08 02:01:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
df_business = spark.read.table("yelp.business")
df_tip = spark.read.table("yelp.tip")
df_checkin = spark.read.table("yelp.checkin")
df_user = spark.read.table("yelp.user")
df_review = spark.read.table("yelp.review")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/05/08 02:01:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
table_list = [df_review, df_user,df_checkin, df_tip, df_business]
# table_list = [ df_user,df_tip, df_review]
# table_list_str = [ 'df_user','df_tip','df_review']
table_list_str = ['df_review', 'df_user','df_checkin', 'df_tip', 'df_business']

In [4]:
table_col_list= [set(i.columns) for i in table_list]

In [5]:
table_col_list

[{'business_id',
  'cool',
  'date',
  'funny',
  'review_id',
  'stars',
  'text',
  'useful',
  'user_id'},
 {'average_stars',
  'compliment_cool',
  'compliment_cute',
  'compliment_funny',
  'compliment_hot',
  'compliment_list',
  'compliment_more',
  'compliment_note',
  'compliment_photos',
  'compliment_plain',
  'compliment_profile',
  'compliment_writer',
  'cool',
  'elite',
  'fans',
  'friends',
  'funny',
  'name',
  'review_count',
  'useful',
  'user_id',
  'yelping_since'},
 {'business_id', 'datetime'},
 {'business_id', 'compliment_count', 'date', 'text', 'user_id'},
 {'address',
  'attributes',
  'business_id',
  'categories',
  'city',
  'hours',
  'is_open',
  'latitude',
  'longitude',
  'name',
  'postal_code',
  'review_count',
  'stars',
  'state'}]

In [6]:
# find the common cols for join key

table_col_dic = dict(zip(table_list_str, table_col_list))

In [7]:
table_col_dic

{'df_review': {'business_id',
  'cool',
  'date',
  'funny',
  'review_id',
  'stars',
  'text',
  'useful',
  'user_id'},
 'df_user': {'average_stars',
  'compliment_cool',
  'compliment_cute',
  'compliment_funny',
  'compliment_hot',
  'compliment_list',
  'compliment_more',
  'compliment_note',
  'compliment_photos',
  'compliment_plain',
  'compliment_profile',
  'compliment_writer',
  'cool',
  'elite',
  'fans',
  'friends',
  'funny',
  'name',
  'review_count',
  'useful',
  'user_id',
  'yelping_since'},
 'df_checkin': {'business_id', 'datetime'},
 'df_tip': {'business_id', 'compliment_count', 'date', 'text', 'user_id'},
 'df_business': {'address',
  'attributes',
  'business_id',
  'categories',
  'city',
  'hours',
  'is_open',
  'latitude',
  'longitude',
  'name',
  'postal_code',
  'review_count',
  'stars',
  'state'}}

In [8]:
table_col_dic.values()

dict_values([{'date', 'stars', 'funny', 'useful', 'cool', 'review_id', 'business_id', 'user_id', 'text'}, {'average_stars', 'fans', 'name', 'compliment_profile', 'compliment_cool', 'compliment_note', 'elite', 'compliment_funny', 'friends', 'compliment_cute', 'review_count', 'yelping_since', 'compliment_list', 'useful', 'compliment_more', 'compliment_plain', 'compliment_writer', 'compliment_hot', 'cool', 'funny', 'compliment_photos', 'user_id'}, {'datetime', 'business_id'}, {'compliment_count', 'date', 'text', 'business_id', 'user_id'}, {'categories', 'longitude', 'name', 'attributes', 'postal_code', 'review_count', 'stars', 'city', 'address', 'state', 'hours', 'is_open', 'latitude', 'business_id'}])

In [9]:
table_col_dic.keys()

dict_keys(['df_review', 'df_user', 'df_checkin', 'df_tip', 'df_business'])

In [10]:
set.intersection(*table_col_dic.values())

set()

In [11]:
# no common joinkey in all tables

In [12]:
for key,values in table_col_dic.items():
    if 'business_id' in values:
        print(key, values
             )

df_review {'date', 'stars', 'funny', 'useful', 'cool', 'review_id', 'business_id', 'user_id', 'text'}
df_checkin {'datetime', 'business_id'}
df_tip {'compliment_count', 'date', 'text', 'business_id', 'user_id'}
df_business {'categories', 'longitude', 'name', 'attributes', 'postal_code', 'review_count', 'stars', 'city', 'address', 'state', 'hours', 'is_open', 'latitude', 'business_id'}


In [13]:
# business_id_table = df_review.join(df_checkin.drop('date'), ['business_id'],'inner')\
#                                    .join(df_tip.drop('date'), ['business_id'],'inner')\
#                                    .join(df_business.drop('date'), ['business_id'],'inner')

In [14]:
# business_id_table.count()

In [15]:
business_id_table1 = df_review.join(df_checkin.drop('date'), ['business_id'],'inner')

business_id_table2 = business_id_table1.join(df_tip.withColumnRenamed('text','tip_text').drop('date'), ['business_id','user_id'],'inner')\
                                   .join(df_business.withColumnRenamed('stars','business_stars').drop('date'), ['business_id'],'inner')




In [16]:
business_id_table1.show(10)


23/05/08 02:02:03 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+-------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|           datetime|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+-------------------+
|znK6tMeegKf9YnGut...|   0|2021-04-16 22:02:21|    0|MmmrqbdupYBJq9jnl...|  5.0|Great little spot...|     0|TjD2S66AWxlKsPSQB...|2010-03-10 00:57:14|
|eVI64EQymywsvMLmD...|   0|2020-11-01 19:50:03|    0|WVAEE47MnM3Sv2cvM...|  5.0|By far the best i...|     0|ZDw_qN5Fy6PE0gdbV...|2010-11-13 02:19:00|
|SPcPJfPgWzhjUDqVF...|   0|2021-03-24 05:53:49|    0|D5b6iooZZcJ8nG-9P...|  5.0|I thought that th...|     0|UvMDlX2wV4Md9OwGW...|2010-09-05 23:15:29|
|T5XzQ6YnVExvd0BOR...|   0|2016-06-13 21:25:10|    0|uDxnT7zWFnxEDxp5e...|  5.0|Aside from the re...

In [17]:
df_tip.show(10)

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|3uLgwr0qeCNMjKenH...|               0|2012-05-18 02:17:21|Avengers time wit...|AGNUgVwnZUey3gcPC...|
|QoezRbYQncpRqyrLH...|               0|2013-02-05 18:35:10|They have lots of...|NBN4MgHP9D3cw--Sn...|
|MYoRNLb5chwjQe3c_...|               0|2013-08-18 00:56:08|It's open even wh...|-copOvldyKh1qr-vz...|
|hV-bABTK-glh5wj31...|               0|2017-06-27 23:05:38|Very decent fried...|FjMQVZjSqY8syIO-5...|
|_uN0OudeJ3Zl_tf6n...|               0|2012-10-06 19:43:09|Appetizers.. plat...|ld0AperBXk1h6Ubqm...|
|7Rm9Ba50bw23KTA8R...|               0|2012-03-13 04:00:52|Chili Cup + Singl...|trf3Qcz8qvCDKXiTg...|
|kH-0iXqkL7b8UXNpg...|               0|2013-12-03 23:42:15|Saturday, Dec 7th...|SM

In [18]:
business_id_table1.count()

6796901

In [19]:
# business_id_table2 = business_id_table1.join(df_tip.withColumnRenamed('text','tip_text').drop('date'), ['user_id'],'inner')
business_id_table2.count()

406315

In [20]:
sorted(business_id_table1.columns)

['business_id',
 'cool',
 'date',
 'datetime',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [21]:
for i in df_user.columns:
    if i != 'user_id':
        df_user = df_user.withColumnRenamed(i, "user_{}".format(i) )

In [22]:
print ('user_uii')

user_uii


In [23]:
print ('user_{}'.format('uii')) 

user_uii


In [24]:
sorted(df_user.columns)

['user_average_stars',
 'user_compliment_cool',
 'user_compliment_cute',
 'user_compliment_funny',
 'user_compliment_hot',
 'user_compliment_list',
 'user_compliment_more',
 'user_compliment_note',
 'user_compliment_photos',
 'user_compliment_plain',
 'user_compliment_profile',
 'user_compliment_writer',
 'user_cool',
 'user_elite',
 'user_fans',
 'user_friends',
 'user_funny',
 'user_id',
 'user_name',
 'user_review_count',
 'user_useful',
 'user_yelping_since']

In [25]:
business_id_table3 = business_id_table2.join(df_user, ['user_id'], 'inner')

In [26]:
df_user.show(10)

+------------------+--------------------+--------------------+---------------------+-------------------+--------------------+--------------------+--------------------+----------------------+---------------------+-----------------------+----------------------+---------+---------------+---------+--------------------+----------+--------------+-----------------+-----------+--------------------+-------------------+
|user_average_stars|user_compliment_cool|user_compliment_cute|user_compliment_funny|user_compliment_hot|user_compliment_list|user_compliment_more|user_compliment_note|user_compliment_photos|user_compliment_plain|user_compliment_profile|user_compliment_writer|user_cool|     user_elite|user_fans|        user_friends|user_funny|     user_name|user_review_count|user_useful|             user_id| user_yelping_since|
+------------------+--------------------+--------------------+---------------------+-------------------+--------------------+--------------------+--------------------+-----

In [27]:
business_id_table3.count()

406315

In [28]:
---STOP---

SyntaxError: invalid syntax (3615926014.py, line 1)

In [30]:
business_id_table3.write.format('parquet').save('gs://data_set_connection/yelp_denormalized_try')

In [ ]:
business_id_table3.count()

In [ ]:
parquet ,    avro 